# Retrieval Augmented Generation(RAG)
- **RAG는 풍부한 정보를 담고 있는 대규모 문서 데이터베이스에서 관련 정보를 검색하고, 이를 통해 언어 모델이 더 정확하고 상세한 답변을 생성**
  - 예)  최신 뉴스 이벤트나 특정 분야의 전문 지식과 같은 주제에 대해 물어보면, RAG는 관련 문서를 찾아 그 내용을 바탕으로 답변을 구성

<br>

### RAG의 8단계 프로세스

<br>

#### 사전 준비단계
1. **도큐먼트 로드 (Document Loader)**: 외부 데이터 소스에서 필요한 문서를 로드하고 초기 처리
2. **텍스트 분할 (Text Splitter)**: 로드된 문서를 처리 가능한 작은 단위로 분할
3. **임베딩 (Embedding)**: 각 문서 또는 문서의 일부를 벡터 형태로 변환하여, 문서의 의미를 수치화
4. **벡터스토어(Vector Store) 저장**: 임베딩된 벡터들을 데이터베이스에 저장

<img src='https://static.wikidocs.net/images/page/233780/rag-graphic-1.png' width=600>


<br>

#### 런타임(RunTime 단계)
1. **검색기 (Retriever)**: 질문이 주어지면, 이와 관련된 벡터를 벡터 데이터베이스에서 검색
2. **프롬프트 (Prompt)**: 검색된 정보를 바탕으로 언어 모델을 위한 질문을 구성. 이는 정보를 바탕으로 어떻게 질문할지 결정하는 과정입니다.
3. **LLM (Large Language Model)**: 구성된 프롬프트를 사용하여 언어 모델이 답변을 생성
4. **체인(Chain) 생성**: 이전의 모든 과정의 하나의 파이프라인으로 묶어주는 체인(Chain) 을 생성

<img src='https://static.wikidocs.net/images/page/233780/rag-graphic-2.png' width=600>

<br>

<hr>

<br>

##  PDF 문서 기반 QA(Question-Answer)
- LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용
  
  $\rightarrow$ 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요
  
  $\rightarrow$ LangSmith를 사용

<br>

### RAG 기본 파이프라인

In [21]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

<br>

#### 1. 문서 로드

In [3]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 23


<br>

#### 2. 문서 분할(Split Documents)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 43


<br>

#### 3. 임베딩(Embedding) 생성

In [5]:
embeddings = OpenAIEmbeddings()

<br>

#### 4. DB 생성(Create DB) 및 저장
- 벡터스토어를 생성

In [6]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

<br>

#### 5. 검색기(Retriever) 생성

In [7]:
retriever = vectorstore.as_retriever()

In [8]:
retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

[Document(id='f634b54c-a0c8-4568-9936-bf881c7f0d30', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\

<br>

#### 6. 프롬프트 생성(Create Prompt) 

In [9]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Question: 
    {question} 
    #Context: 
    {context} 

    #Answer:"""
)

<br>

#### 7. 언어모델(LLM) 생성

In [10]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

<br>

#### 8. 체인(Chain) 생성

In [11]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

- 체인에 쿼리(질문)을 입력하고 실행

In [12]:
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

삼성전자가 자체 개발한 AI의 이름은 '삼성 가우스'입니다.


<br>

<hr>

<br>

## 네이버 뉴스 기반 QA(Question-Answering) 챗봇


In [41]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

<br>

#### 1. 문서 로드 (크롤링)

In [23]:
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

<SoupStrainer name=[<TagNameMatchRule string=div pattern=None function=None present=None>] attrs=defaultdict(<class 'list'>, {'class': [<AttributeValueMatchRule string=newsct_article _article_body pattern=None function=None present=None>, <AttributeValueMatchRule string=media_end_head_title pattern=None function=None present=None>]}) string=[]>

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")

In [26]:
print(docs[0].page_content[:500])


출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책


[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨


<br>

#### 2. 문서 분할

In [50]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

<br>

#### 3. 임베딩 & 벡터 스토어 & 리트리버 생성

In [51]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

<br>

#### 4. 프롴프트 생성

In [52]:
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
    검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
    한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

    #Question: 
    {question} 

    #Context: 
    {context} 

    #Answer:"""
)

<br>

#### 5. 언어 모델

In [59]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

<br>

#### 6. 체인 생성

In [60]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [62]:
print(rag_chain.invoke("부영그룹의 출산 장려 정책에 대해 설명해주세요."))

부영그룹의 출산 장려 정책은 다음과 같습니다. 2021년 이후 태어난 직원 자녀에게 1억원씩, 총 70억원을 지원하며, 이 정책을 앞으로도 이어갈 계획입니다. 연년생과 쌍둥이 자녀가 있는 경우 총 2억원을 받을 수 있습니다. 또한, 셋째 아이를 낳는 경우 국민주택을 제공하겠다는 계획도 밝혔습니다. 이러한 정책은 출산한 직원에게 금전적인 지원을 통해 경제적 부담을 덜어주고, 출산을 장려하기 위한 것입니다.


<br>

<hr>